# FinBERT Profiling: Mini FinBert + Knowledge Distillation

This notebook is intentionally **thin**: it reuses the profiling utilities in `pipelines/finBERT/finbert/` (especially `finbert/finbert_profile.py` and `finbert/profile_utils.py`) instead of copying large code blocks.

The purpose of this notebook is to profile finetuning and distilling a 'mini-bert' model on financial sentiment analysis.


In [1]:
from __future__ import annotations

from pathlib import Path
import shutil
import os
import logging
import sys
sys.path.append('..')

from textblob import TextBlob
from pprint import pprint
from sklearn.metrics import classification_report

from transformers import AutoModelForSequenceClassification

from finbert.finbert import *
from finbert.finbert_profile import *
from finbert.profile_utils import get_model_size_mb, print_device_info, setup_nltk_data, timed_eval
import finbert.utils as tools

from finbert.finbert_kd_trainer import KDFinBert



%load_ext autoreload
%autoreload 2

project_dir = Path.cwd().parent
pd.set_option('max_colwidth', None)

import wandb

✓ Helper utilities loaded


In [2]:
wandb.init(
    entity="si2449-columbia-university",
    project="Project-Runs",
    name="finetuning/distilling-mini_bert",
    group="knowledge-distillation",
)

wandb: Currently logged in as: shriya-mahakala (aqlab) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [3]:
cl_path = project_dir/'models'/'student'
cl_data_path = project_dir/'data'/'sentiment_data'

In [4]:
try:
    shutil.rmtree(cl_path) 
except:
    pass

bertmodel = AutoModelForSequenceClassification.from_pretrained("prajjwal1/bert-mini", cache_dir=None, num_labels=3)


config = Config(   data_dir=cl_data_path,
                   bert_model=bertmodel,
                   num_train_epochs=6,
                   model_dir=cl_path,
                   max_seq_length = 64,
                   train_batch_size = 32,
                   learning_rate = 0.00001420326287435756,
                   output_mode='classification',
                   warm_up_proportion=0.14386028719686458,
                   local_rank=-1,
                   discriminate=True,
                   gradual_unfreeze=False,
                   encoder_no = 4)

config.profile_train_steps = 20
config.num_hidden_layers = 4

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at prajjwal1/bert-mini and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
finbert = ProfiledFinBert(config)
finbert.config.base_model = 'prajjwal1/bert-mini'
finbert.config.discriminate=True
finbert.config.gradual_unfreeze=False

In [6]:
finbert.prepare_model(label_list=['positive','negative','neutral'])

12/17/2025 03:02:39 - INFO - finbert.finbert -   device: cuda n_gpu: 1, distributed training: False, 16-bits training: False


In [7]:
train_data = finbert.get_data('train')

In [8]:
model = finbert.create_the_model()

In [9]:
start = time.perf_counter()
trained_model = finbert.train(train_examples = train_data, model = model)
train_wall_s = time.perf_counter() - start

12/17/2025 03:02:42 - INFO - finbert.utils -   *** Example ***
12/17/2025 03:02:42 - INFO - finbert.utils -   guid: train-1
12/17/2025 03:02:42 - INFO - finbert.utils -   tokens: [CLS] after the reporting period , bio ##tie north american licensing partner so ##max ##on pharmaceuticals announced positive results with na ##lm ##efe ##ne in a pilot phase 2 clinical trial for smoking ce ##ssa ##tion [SEP]
12/17/2025 03:02:42 - INFO - finbert.utils -   input_ids: 101 2044 1996 7316 2558 1010 16012 9515 2167 2137 13202 4256 2061 17848 2239 24797 2623 3893 3463 2007 6583 13728 27235 2638 1999 1037 4405 4403 1016 6612 3979 2005 9422 8292 11488 3508 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
12/17/2025 03:02:42 - INFO - finbert.utils -   attention_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
12/17/2025 03:02:42 - INFO - finbert.utils -   token_type_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

12/17/2025 03:02:42 - INFO - finbert.finbert -   ***** Loading data *****
12/17/2025 03:02:42 - INFO - finbert.finbert -     Num examples = 3488
12/17/2025 03:02:42 - INFO - finbert.finbert -     Batch size = 32
12/17/2025 03:02:42 - INFO - finbert.finbert -     Num steps = 72



Starting Profiled Training
Device: cuda
Profiling activities: [<ProfilerActivity.CPU: 0>, <ProfilerActivity.CUDA: 2>]



Epoch:   0%|          | 0/6 [00:01<?, ?it/s]



Profiling complete for first epoch (20 steps)
Continuing full training without profiling...


PROFILING RESULTS - Training


By CPU Time:
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg       CPU Mem  Self CPU Mem      CUDA Mem  Self CUDA Mem    # of Calls  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                       cudaLaunchKernel         7.29%      89.383ms        27.72%

...

Validation losses: [1.0473814927614653, 0.9731460855557368, 0.9198884413792536, 0.8852779039969811]


Iteration: 100%|██████████| 109/109 [00:02<00:00, 45.44it/s]
12/17/2025 03:03:13 - INFO - finbert.utils -   *** Example ***
12/17/2025 03:03:13 - INFO - finbert.utils -   guid: validation-1
12/17/2025 03:03:13 - INFO - finbert.utils -   tokens: [CLS] our in - depth expertise extends to the fields of energy , industry , urban & mobility and water & environment [SEP]
12/17/2025 03:03:13 - INFO - finbert.utils -   input_ids: 101 2256 1999 1011 5995 11532 8908 2000 1996 4249 1997 2943 1010 3068 1010 3923 1004 12969 1998 2300 1004 4044 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
12/17/2025 03:03:13 - INFO - finbert.utils -   attention_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
12/17/2025 03:03:13 - INFO - finbert.utils -   token_type_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

Validation losses: [1.0473814927614653, 0.9731460855557368, 0.9198884413792536, 0.8852779039969811, 0.8637106602008526]


Iteration: 100%|██████████| 109/109 [00:02<00:00, 45.27it/s]
12/17/2025 03:03:15 - INFO - finbert.utils -   *** Example ***
12/17/2025 03:03:15 - INFO - finbert.utils -   guid: validation-1
12/17/2025 03:03:15 - INFO - finbert.utils -   tokens: [CLS] our in - depth expertise extends to the fields of energy , industry , urban & mobility and water & environment [SEP]
12/17/2025 03:03:15 - INFO - finbert.utils -   input_ids: 101 2256 1999 1011 5995 11532 8908 2000 1996 4249 1997 2943 1010 3068 1010 3923 1004 12969 1998 2300 1004 4044 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
12/17/2025 03:03:15 - INFO - finbert.utils -   attention_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
12/17/2025 03:03:15 - INFO - finbert.utils -   token_type_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

Validation losses: [1.0473814927614653, 0.9731460855557368, 0.9198884413792536, 0.8852779039969811, 0.8637106602008526, 0.8555622054980352]


In [10]:
test_data = finbert.get_data("test")

results = finbert.evaluate(examples=test_data, model=trained_model)

eval_df, eval_timing = timed_eval(
    finbert=finbert, model=trained_model, examples=test_data, use_amp=False
)

12/17/2025 03:03:16 - INFO - finbert.utils -   *** Example ***
12/17/2025 03:03:16 - INFO - finbert.utils -   guid: test-1
12/17/2025 03:03:16 - INFO - finbert.utils -   tokens: [CLS] the bristol port company has sealed a one million pound contract with cooper specialised handling to supply it with four 45 - ton ##ne , custom ##ised reach stack ##ers from ko ##ne ##cr ##ane ##s [SEP]
12/17/2025 03:03:16 - INFO - finbert.utils -   input_ids: 101 1996 7067 3417 2194 2038 10203 1037 2028 2454 9044 3206 2007 6201 17009 8304 2000 4425 2009 2007 2176 3429 1011 10228 2638 1010 7661 5084 3362 9991 2545 2013 12849 2638 26775 7231 2015 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
12/17/2025 03:03:16 - INFO - finbert.utils -   attention_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
12/17/2025 03:03:16 - INFO - finbert.utils -   token_type_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

Testing: 100%|██████████| 31/31 [00:00<00:00, 151.65it/s]
12/17/2025 03:03:16 - INFO - finbert.utils -   *** Example ***
12/17/2025 03:03:16 - INFO - finbert.utils -   guid: test-1
12/17/2025 03:03:16 - INFO - finbert.utils -   tokens: [CLS] the bristol port company has sealed a one million pound contract with cooper specialised handling to supply it with four 45 - ton ##ne , custom ##ised reach stack ##ers from ko ##ne ##cr ##ane ##s [SEP]
12/17/2025 03:03:16 - INFO - finbert.utils -   input_ids: 101 1996 7067 3417 2194 2038 10203 1037 2028 2454 9044 3206 2007 6201 17009 8304 2000 4425 2009 2007 2176 3429 1011 10228 2638 1010 7661 5084 3362 9991 2545 2013 12849 2638 26775 7231 2015 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
12/17/2025 03:03:16 - INFO - finbert.utils -   attention_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
12/17/2025 03:03:16 - INFO - finbert.utils -   token_type_ids

In [11]:
def report(df, cols=['label','prediction','logits']):
    #print('Validation loss:{0:.2f}'.format(metrics['best_validation_loss']))
    cs = CrossEntropyLoss(weight=finbert.class_weights)
    loss = cs(torch.tensor(list(df[cols[2]])),torch.tensor(list(df[cols[0]])))
    print("Evaluation Loss:{0:.2f}".format(loss))
    print("Evaluation Accuracy:{0:.2f}".format((df[cols[0]] == df[cols[1]]).sum() / df.shape[0]) )
    print("\nClassification Report:")
    return_val = classification_report(df[cols[0]], df[cols[1]], output_dict=True)
    
    
    new_report = {}
    
    for key in return_val.keys():
        new_report["Finetuning " + key] = return_val[key]
    
    new_report["Finetuning Evalaution Loss"] = loss
    new_report["Fintuning Evaluation Accuracy"] = (df[cols[0]] == df[cols[1]]).sum() / df.shape[0]
    
    
    print(new_report)
        
    return new_report

In [12]:
results['prediction'] = results.predictions.apply(lambda x: np.argmax(x,axis=0))

In [13]:
wandb_report = report(results,cols=['labels','prediction','predictions'])


Evaluation Loss:0.89
Evaluation Accuracy:0.64

Classification Report:
{'Finetuning 0': {'precision': 0.4666666666666667, 'recall': 0.26217228464419473, 'f1-score': 0.33573141486810554, 'support': 267.0}, 'Finetuning 1': {'precision': 0.3835616438356164, 'recall': 0.65625, 'f1-score': 0.484149855907781, 'support': 128.0}, 'Finetuning 2': {'precision': 0.778702163061564, 'recall': 0.8139130434782609, 'f1-score': 0.7959183673469388, 'support': 575.0}, 'Finetuning accuracy': 0.6412371134020619, 'Finetuning macro avg': {'precision': 0.5429768245212824, 'recall': 0.5774451093741518, 'f1-score': 0.5385998793742751, 'support': 970.0}, 'Finetuning weighted avg': {'precision': 0.6406697259498538, 'recall': 0.6412371134020619, 'f1-score': 0.6281077634540928, 'support': 970.0}, 'Finetuning Evalaution Loss': tensor(0.8872), 'Fintuning Evaluation Accuracy': np.float64(0.6412371134020619)}


/var/tmp/ipykernel_173151/142971867.py:4: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /pytorch/torch/csrc/utils/tensor_new.cpp:253.)
  loss = cs(torch.tensor(list(df[cols[2]])),torch.tensor(list(df[cols[0]])))


In [14]:
teacher_path = project_dir/'models'/'teacher'
teacher = AutoModelForSequenceClassification.from_pretrained(
    teacher_path, num_labels=3, cache_dir=None
)
student = trained_model

new_path = project_dir/'models'/'distilled_student'

In [15]:
config = Config(   data_dir=cl_data_path,
                   bert_model=None,
                   num_train_epochs=2,
                   model_dir=new_path,
                   max_seq_length = 48,
                   train_batch_size = 32,
                   learning_rate = 5e-6,
                   output_mode='classification',
                   warm_up_proportion=0.1,
                   local_rank=-1,
                   discriminate=True,
                   gradual_unfreeze=True,
                   encoder_no = 4,
                   base_model='prajjwal1/bert-mini')


In [16]:
kd = KDFinBert(teacher=teacher, student=student, config=config)

In [17]:
kd.prepare_model(label_list=["positive", "negative", "neutral"])

In [18]:
train_data = kd.get_data('train')
model = kd.create_the_model()

In [19]:
start = time.perf_counter()
trained_model = kd.train(train_examples = train_data, teacher=teacher, student=student)
train_wall_s = time.perf_counter() - start

12/17/2025 03:03:21 - INFO - finbert.utils -   *** Example ***
12/17/2025 03:03:21 - INFO - finbert.utils -   guid: train-1
12/17/2025 03:03:21 - INFO - finbert.utils -   tokens: [CLS] after the reporting period , bio ##tie north american licensing partner so ##max ##on pharmaceuticals announced positive results with na ##lm ##efe ##ne in a pilot phase 2 clinical trial for smoking ce ##ssa ##tion [SEP]
12/17/2025 03:03:21 - INFO - finbert.utils -   input_ids: 101 2044 1996 7316 2558 1010 16012 9515 2167 2137 13202 4256 2061 17848 2239 24797 2623 3893 3463 2007 6583 13728 27235 2638 1999 1037 4405 4403 1016 6612 3979 2005 9422 8292 11488 3508 102 0 0 0 0 0 0 0 0 0 0 0
12/17/2025 03:03:21 - INFO - finbert.utils -   attention_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0
12/17/2025 03:03:21 - INFO - finbert.utils -   token_type_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
12/17/

12/17/2025 03:03:21 - INFO - finbert.finbert_kd_trainer -   ***** Loading data *****
12/17/2025 03:03:21 - INFO - finbert.finbert_kd_trainer -     Num examples = 3488
12/17/2025 03:03:21 - INFO - finbert.finbert_kd_trainer -     Batch size = 32
12/17/2025 03:03:21 - INFO - finbert.finbert_kd_trainer -     Num steps = 24
/home/srm2245/hpml-project/pipelines/finBERT/notebooks/../finbert/finbert_kd_trainer.py:314: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=self.config.use_amp)
Epoch:   0%|          | 0/2 [00:00<?, ?it/s]/home/srm2245/hpml-project/pipelines/finBERT/notebooks/../finbert/finbert_kd_trainer.py:356: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=self.config.use_amp):
Epoch:   0%|          | 0/2 [00:00<?, ?it/s]/home/srm2245/hpml-project/

Validation losses: [0.8395374967501714]


Iteration: 100%|██████████| 109/109 [00:11<00:00,  9.81it/s]
12/17/2025 03:04:00 - INFO - finbert.utils -   *** Example ***
12/17/2025 03:04:00 - INFO - finbert.utils -   guid: validation-1
12/17/2025 03:04:00 - INFO - finbert.utils -   tokens: [CLS] our in - depth expertise extends to the fields of energy , industry , urban & mobility and water & environment [SEP]
12/17/2025 03:04:00 - INFO - finbert.utils -   input_ids: 101 2256 1999 1011 5995 11532 8908 2000 1996 4249 1997 2943 1010 3068 1010 3923 1004 12969 1998 2300 1004 4044 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
12/17/2025 03:04:00 - INFO - finbert.utils -   attention_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
12/17/2025 03:04:00 - INFO - finbert.utils -   token_type_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
12/17/2025 03:04:00 - INFO - finbert.utils -   label: neutral (id = 2)
12/17/2025 03:04:00 

Validation losses: [0.8395374967501714, 0.8252754394824688]


In [20]:
test_data = kd.get_data("test")

results = kd.evaluate(examples=test_data, model=trained_model)



12/17/2025 03:04:01 - INFO - finbert.utils -   *** Example ***
12/17/2025 03:04:01 - INFO - finbert.utils -   guid: test-1
12/17/2025 03:04:01 - INFO - finbert.utils -   tokens: [CLS] the bristol port company has sealed a one million pound contract with cooper specialised handling to supply it with four 45 - ton ##ne , custom ##ised reach stack ##ers from ko ##ne ##cr ##ane ##s [SEP]
12/17/2025 03:04:01 - INFO - finbert.utils -   input_ids: 101 1996 7067 3417 2194 2038 10203 1037 2028 2454 9044 3206 2007 6201 17009 8304 2000 4425 2009 2007 2176 3429 1011 10228 2638 1010 7661 5084 3362 9991 2545 2013 12849 2638 26775 7231 2015 102 0 0 0 0 0 0 0 0 0 0
12/17/2025 03:04:01 - INFO - finbert.utils -   attention_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0
12/17/2025 03:04:01 - INFO - finbert.utils -   token_type_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
12/17/2025 03:04:01 - IN

12/17/2025 03:04:01 - INFO - finbert.finbert_kd_trainer -     Batch size = 32
12/17/2025 03:04:01 - INFO - finbert.finbert_kd_trainer -     Num steps = 60
12/17/2025 03:04:01 - INFO - finbert.finbert_kd_trainer -   ***** Running evaluation ***** 
12/17/2025 03:04:01 - INFO - finbert.finbert_kd_trainer -     Num examples = 970
12/17/2025 03:04:01 - INFO - finbert.finbert_kd_trainer -     Batch size = 32
Testing: 100%|██████████| 31/31 [00:00<00:00, 178.27it/s]


In [21]:
results['prediction'] = results.predictions.apply(lambda x: np.argmax(x,axis=0))

In [22]:
def kd_report(df, cols=['label','prediction','logits']):
    #print('Validation loss:{0:.2f}'.format(metrics['best_validation_loss']))
    cs = CrossEntropyLoss(weight=finbert.class_weights)
    loss = cs(torch.tensor(list(df[cols[2]])),torch.tensor(list(df[cols[0]])))
    print("Loss:{0:.2f}".format(loss))
    print("Accuracy:{0:.2f}".format((df[cols[0]] == df[cols[1]]).sum() / df.shape[0]) )
    print("\nClassification Report:")
    return_val = classification_report(df[cols[0]], df[cols[1]], output_dict=True)
    
    return_val["Evaluation Loss"] = loss
    return_val["Evaluation Accuracy"] = (df[cols[0]] == df[cols[1]]).sum() / df.shape[0]
    return return_val

In [23]:
wandb_kd_report = kd_report(results,cols=['labels','prediction','predictions'])

summary = {
        "device": str(finbert.device),
        "model_dir": str(cl_path),
        "train_wall_s": float(train_wall_s),
        "train_examples": int(len(train_data)),
        "train_examples_per_s": float((len(train_data) * finbert.config.num_train_epochs) / train_wall_s)
        if train_wall_s > 0
        else float("inf"),
        "model_size_mb": float(get_model_size_mb(trained_model)),
        "profile_train_steps": finbert.config.profile_train_steps,
        **(finbert.profile_results.get("training_summary", {}) or {}),
        **eval_timing,
        **wandb_report,
        **wandb_kd_report,
    }

wandb.log(summary)
wandb.finish()

Loss:0.86
Accuracy:0.69

Classification Report:


Evaluation Accuracy,▁
Evaluation Loss,▁
Finetuning Evalaution Loss,▁
Finetuning accuracy,▁
Fintuning Evaluation Accuracy,▁
accuracy,▁
epoch,▁▁▁▁▁▁▁▁▂▂▂▄▄▄▄▄▄▄▄▅▅▅▅▅▅▅▇▇▇▇▇▇▇▇██████
eval_num_samples,▁
eval_samples_per_s,▁
eval_wall_s,▁
+14,...
